In [ ]:
## Human hg19 feather file (500bp up + 10 bp centered); module genes>20

In [1]:
import os
import glob
import pickle
import pandas as pd
import numpy as np

from dask.diagnostics import ProgressBar

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2

from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies, load_motifs
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

import seaborn as sns

In [2]:
DATA_FOLDER="/data/WWT_TYY_CD34/SCENIC_20230831"
RESOURCES_FOLDER="/data/Scenic_Python_ResourceFolder"
DATABASE_FOLDER = "/data/Scenic_Python_ResourceFolder/Human_RcisTarget_Database"
SCHEDULER="192.168.1.200:8888"
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "hg19-*.mc9nr.genes_vs_motifs.rankings.feather")
MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDER, "motifs-v9-nr.hgnc-m0.001-o0.0.tbl")
MM_TFS_FNAME = os.path.join(RESOURCES_FOLDER, 'hs_hgnc_tfs.txt')
SC_EXP_FNAME = os.path.join(RESOURCES_FOLDER, "CD34_exp.csv")
REGULONS_FNAME = os.path.join(DATA_FOLDER, "regulons.p")
MOTIFS_FNAME = os.path.join(DATA_FOLDER, "motifs.csv")

In [3]:
ex_matrix = pd.read_csv(SC_EXP_FNAME, sep=',', header=0, index_col=0).T
ex_matrix.shape

(13195, 16111)

In [4]:
tf_names = load_tf_names(MM_TFS_FNAME)

In [5]:
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs

[]

In [6]:
adjacencies = grnboost2(ex_matrix, tf_names=tf_names, verbose=True)

preparing dask client


/home/gaoyuchen/.local/lib/python3.7/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40238 instead
  f"Port {expected} is already in use.\n"


parsing input
creating dask graph
8 partitions
computing dask graph
shutting down client and local cluster
finished


In [7]:
adjacencies
adjacencies.to_csv('/data/WWT_TYY_CD34/SCENIC_20230831/adjacencies.csv')
## adjacencies = pd.read_csv("adj.tsv", index_col=False, sep='\t')

In [8]:
modules = list(modules_from_adjacencies(adjacencies, ex_matrix, min_genes=10))
##each module min gene =20 below 20 will remove; as default


2023-09-01 21:08:05,383 - pyscenic.utils - INFO - Calculating Pearson correlations.

2023-09-01 21:08:07,022 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [False].

2023-09-01 21:09:02,552 - pyscenic.utils - INFO - Creating modules.


In [12]:
## need to convert feather file first time
import argparse
from typing import Optional

import pyarrow.compute as pc
import pyarrow.feather as pf

In [13]:
def convert_feather_v1_to_v2_vice_versa(
        input_ct_db_filename: str,
        output_ct_db_filename: str,
        compression: Optional[str] = "zstd",
        compression_level: int = 6,
        to_version: int = 2,
):
    """
    Convert cisTarget Feather database from Feather v1 to v2 format (with or without compression) and vice versa.
    :param input_ct_db_filename: input cisTarget database filename.
    :param output_ct_db_filename: output cisTarget database filename.
    :param compression: Compression method: "zstd" (default), "lz4" or "uncompressed".
    :param compression_level: Compression level for "zstd" or "lz4".
    :param to_version: Output Feather file format version: 1 (legacy) or 2 (default).
    :return:
    """

    if to_version != 2 and to_version != 1:
        raise ValueError("Feather file version only supports 1 (legacy) or 2 (default).")

    if to_version == 1:
        # Compression is not supported in Feather v1 format.
        compression = "uncompressed"
        compression_level = None

    if compression not in {"zstd", "lz4", "uncompressed"}:
        raise ValueError(
            f'Unsupported compression value "{compression}". Choose "zstd" (default), "lz4" or "uncompressed".'
        )

    # Read input cisTarget database as a pyarrow Table.
    df_pa_table = pf.read_table(
        source=input_ct_db_filename,
    )

    # Get all column names.
    all_column_names = df_pa_table.column_names

    try:
        # Check if we have an old database that still used a "features" column and rename it.
        features_idx = all_column_names.index("features")

        # Get column which contains motif or track names.
        motifs_or_track_names = df_pa_table.column(features_idx)

        if pc.sum(pc.starts_with(motifs_or_track_names, "jaspar")).as_py() > 0:
            # It is a motif vs genes/regions database if JASPAR motif names were found in the "features" column.
            all_column_names[features_idx] = "motifs"
        else:
            all_column_names[features_idx] = "tracks"

        df_pa_table.drop(["features"])
        # Rename features column in database to "motifs" or "tracks".
        df_pa_table = df_pa_table.rename_columns(all_column_names)
    except ValueError:
        # No old database (with "features" column).
        pass

    # Get database index column ("motifs", "tracks", "regions" or "genes" depending of the database type).
    for column_idx, column_name in enumerate(all_column_names):
        if column_name in {"motifs", "tracks", "regions", "genes"}:
            index_column = df_pa_table.column(column_idx)
            break

    # Sort column names (non-index columns) and add index column as last column.
    column_names_sorted_and_index = sorted(
        [
            column_name
            for column_name in all_column_names
            if column_name not in index_column._name
        ]
    )
    column_names_sorted_and_index.append(index_column._name)

    # Create a new pyarrow Table with columns in the new order.
    df_pa_table = df_pa_table.select(column_names_sorted_and_index)

    # Writhe cisTarget database to a new Feather file with the requested compression/version settings.
    pf.write_feather(
        df=df_pa_table,
        dest=output_ct_db_filename,
        compression=compression,
        compression_level=compression_level,
        version=to_version
    )


In [15]:
convert_feather_v1_to_v2_vice_versa(input_ct_db_filename='/data/Scenic_Python_ResourceFolder/Human_RcisTarget_Database/hg19-500bp-upstream-10species.mc9nr.feather',
                                    output_ct_db_filename='/data/Scenic_Python_ResourceFolder/Human_RcisTarget_Database/hg19-500bp-upstream-10species.mc9nr.genes_vs_motifs.rankings.feather',to_version=2,
                                   compression_level=6)

In [16]:
convert_feather_v1_to_v2_vice_versa(input_ct_db_filename='/data/Scenic_Python_ResourceFolder/Human_RcisTarget_Database/hg19-tss-centered-10kb-10species.mc9nr.feather',
                                    output_ct_db_filename='/data/Scenic_Python_ResourceFolder/Human_RcisTarget_Database/hg19-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings.feather',to_version=2,
                                   compression_level=6)

In [17]:
##try again
## From the Scenic website: By default, SCENIC uses the databases that score the motifs in the promoter of the genes (up to 500bp upstream the TSS), and in the 20kb around the TSS (+/-10kbp).
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs

[FeatherRankingDatabase(name="hg19-500bp-upstream-10species.mc9nr.genes_vs_motifs.rankings"),
 FeatherRankingDatabase(name="hg19-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings")]

In [18]:
# Calculate a list of enriched motifs and the corresponding target genes for all modules.
with ProgressBar():
    df = prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME, client_or_address='dask_multiprocessing')

# Create regulons from this table of enriched motifs.
regulons = df2regulons(df)

# Save the enriched motifs and the discovered regulons to disk.
df.to_csv(MOTIFS_FNAME)
with open(REGULONS_FNAME, "wb") as f:
    pickle.dump(regulons, f)

[                                        ] | 0% Completed |  4min 41.1s


2023-09-01 22:50:20,976 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for ZNF606 could be mapped to hg19-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.


[                                        ] | 0% Completed |  4min 41.8s


2023-09-01 22:50:21,638 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for MESP2 could be mapped to hg19-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.


[                                        ] | 0% Completed |  4min 42.4s

...

[######################################  ] | 96% Completed | 17min 42.8s


2023-09-01 23:03:22,708 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for POU4F1 could be mapped to hg19-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.


[######################################  ] | 96% Completed | 17min 48.9s


2023-09-01 23:03:28,757 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for PRDM16 could be mapped to hg19-tss-centered-10kb-10species.mc9nr.genes_vs_motifs.rankings. Skipping this module.


[########################################] | 100% Completed | 18min 14.2s
Create regulons from a dataframe of enriched features.
Additional columns saved: []


In [19]:
auc_mtx = aucell(ex_matrix, regulons, num_workers=4)

In [20]:
auc_mtx.to_csv('/data/WWT_TYY_CD34/SCENIC_20230831/auc_Score.csv')